# 第4章 因果推論を実装しよう

#### ・本章では、因果の大きさを推定する、すなわち因果推論を実施する手法として、回帰分析、IPTW法、DR法について解説、実装する .

## 4-1 回帰分析による因果推論の実装

## 回帰モデルの導出

#### ・回帰分析による因果推論では、入力変数を与えたときに出力変数の値を求めるモデルを構築する .

#### ・回帰モデルの入力変数 : バックドアパスを閉じるように因果ダイアグラムをd分離したあとに残っている変数のみを使用 .

#### ・回帰モデルの出力変数 : 因果効果が表れる変数 .

#### ・回帰分析による因果推論を英語では regression adjustment と呼ぶ .

#### ・因果推論したいモデルとして、以下のテレビCM効果の架空事例を想定し、この因果ダイアグラムからテレビCM効果を推定する .

![alt text](pict1.png)

#### ・図4.1.1の年齢を変数$x_1$、性別を変数$x_2$として表し、とある「iさん」の年齢と性別は、$x^i_1$ , $x^i_2$と表すことにする .

#### ・とある「iさん」が[テレビCMを見た]かどうか、すなわち処置を受けたかどうかを$Z^i$と記載する .

#### テレビCMを見た場合は $Z^i$ = 1 となり、$Z^i_1$と記載する .　テレビCMを見ていない場合は $Z^i$ = 0 となり、$Z^i_0$と記載する .

#### この変数$Z$のように、処置を受けたなどを示す変数は"原因変数"とも呼ばれる .

#### ・次に、とある「iさん」の商品の購入量(連続変数)を$Y^i$と記載する .

#### $Z^i$ = 1 の場合は$Y^i_1$と記載し、$Z^i$ = 0 の場合は$Y^i_0$と記載する .

#### $Y$のような、処置による結果が現れる変数は"結果変数"とも呼ばれる .

#### ・図4.1.1では年齢変数$x_1$と性別変数$x_2$は、変数$Z$と変数$Y$の間に疑似相関を生む交絡因子(共通因子)である .　そのためd分離するには変数$x_1$と変数$x_2$を考慮する .

#### これは感覚的には、d分離した後に残っている変数(共変量)を考慮して、結果変数を求めるという操作である .

#### ・この場合、具体的には以下のような2つの式を立てる .

$$
Y^i_1 = f_1(Z^i,x^i_1,x^i_2)
$$
$$
Y^i_0 = f_0(Z^i,x^i_1,x^i_2)
$$

#### ・ここで、本説の例では結果変数$Y$が連続値なので、上記の$f()$には線型回帰を利用する .

#### 結果変数$Y$が離散値(カテゴリー変数)であった場合にはロジスティック回帰を使用する .

#### ・ここで、性別を示す変数$x_2$はカテゴリー変数であり、女性の場合を0、男性の場合を１とする .

#### 年齢を示す変数$x_1$は連続的な整数値となる .

#### ・線型回帰モデルの形で上式を書き直すと、下のようになる .

$$
Y^i_1 = b_{z1}Z^i_1 + b_1x^i_1 + b_2x^i_2 + b_0 = b_z × 1 + b_1x^i_1 + b_2x^i_2 + b_0
$$
$$
Y^i_0 = b_{z0}Z^i_0 + b_1x^i_1 + b_2x^i_2 + b_0 = b_z × 0 + b_1x^i_1 + b_2x^i_2 + b_0
$$

#### ここで、変数$b$は各係数を示し、$b_0$はバイアス項である .　上記において処置の変数$Z$は0か1の2値であり、$b_{z0}$にかかる$Z^i_0$が0より、$b_{z1}$ = $b_{z0}$ = $b_z$ と書いても問題ない .

#### ・その結果、関数$f_0$と関数$f_1$は同一になり、回帰モデルは最終的に以下のようになる .

$$
Y^i = b_zZ^i + b_1x^i_1 + b_2x^i_2 + b_0
$$

#### あとは実際のデータからこのモデルの係数$b_z$,$b_1$,$b_2$,$b_0$を求めればよい .

## 回帰モデルの実装

#### ・本節では、テレビCM効果を確かめる疑似ケースの実装を行う .

#### 回帰モデルを求める際に、Pythonの機械学習ライブラリであるscikit-learnを使用する .

In [1]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [3]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd

#### ・はじめに疑似データを作成する .

#### 年齢変数$x_1$は15歳から75歳の一様分布に従うとする .　性別変数$x_2$は0を女性、1を男性とし、50%の確率で男性か女性とする .

In [4]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数

#### ・次にテレビCMを見たかどうかを決める .

#### 年齢$x_1$が高いほど、そして性別$x_2$が

## 回帰モデルの補足

#### ・以下に示す図4.1.3のように、仮に結果変数$Y$にだけ効く別の変数$a$が観測できている場合、d分離する際に変数$a$を残す必要はない .

#### つまり、変数$a$は疑似相関、間接的因果を生み出していないが、変数$Y$の正確な回帰モデルを構築するために回帰モデルの入力変数として考慮する方が正確な因果推論が可能になる .

![alt text](pict2.png)

## 4-2 傾向スコアを用いた逆確率重み付け法(IPTW)の実装

#### ・傾向スコアと呼ばれる指標を使用した、逆確率重み付け法(IPTW : Inverse Probability of Treatment Weighting もしくは IPW)を解説・実装する .

## 傾向スコアとは

#### ・傾向スコア(propensity score) : 処置を受ける確率に類する値、すなわち「処置を受ける傾向」を示す指標 .

#### ・IPTW法 : 測定されたデータに対して、傾向スコアで結果を調整して、因果の効果を推定する手法 .

#### ・はじめに傾向スコアについて解説する .　ここで、調整化公式を再掲する .

$$
P(Y = y|do(Z = z)) = Σ_xP(Y = y|Z = z,X = x)P(X = x)
$$

#### ・この調整化公式の右辺は2つの項、$P(Y = y|Z = z,X = x)$ と P(X = x)の掛け算となっていて、取り扱いにくいという嫌な点がある .